<a href="https://colab.research.google.com/github/bbarber314/MATH527_MLForFinance/blob/main/527Hw2Prob3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider a feedforward neural network with three inputs, two units in the first hidden layer, two units in the second hidden layer, and three units in the output layer. The activation function for hidden layer 1 is ReLU, for hidden layer 2 is sigmoid, and for the output layer is softmax.

The initial weights are given by the matrices
$W^{(1)} =\begin{pmatrix}0.1&0.3&0.7\\0.9& 0.4& 0.4\end{pmatrix}$ , $W^{(2)} =\begin{pmatrix}0.4&0.3\\0.7&0.2\end{pmatrix}$, $W^{(3)} =\begin{pmatrix}0.5&0.6\\0.6& 0.7\\0.3& 0.2\end{pmatrix}$,
and all the biases are unit vectors.

Assuming that the input $\begin{pmatrix}0.1\\0.7\\0.3\end{pmatrix}$
corresponds to the output $\begin{pmatrix}1\\0\\0\end{pmatrix}$ , manually compute the updated weights and biases after a single epoch (forward + backward pass), clearly stating all derivatives that you have used. You should use a learning rate of 1.

As a practical exercise, you should modify the implementation of a stochastic gradient descent routine in the back-propagation Python notebook. ML_in_Finance-Backpropagation.ipynb. Note that the notebook example corresponds to the example in Chapter 4 of the textbook (and shown in class 1), which uses sigmoid activated hidden layers only. Compare the weights and biases obtained by TensorFlow (or your ANN library of choice) with those obtained by your procedure after 200 epoch

Solution:

In [ ]:
import numpy as np 
from numpy.linalg import norm

import copy
import os

def relu(x):
  return x*(np.sign(x)+1.)/2.

def sigmoid(x):
  return 1./(1.+np.exp(-x))

def softmax(x):
  return np.exp(x)/sum(np.exp(x))

In [ ]:
W0_1 = np.array([[0.1,0.3,0.7], [0.9,0.4,0.4]])
b_1 = np.array([1.,1.])

W0_2 = np.array([[0.4,0.3], [0.7,0.2]])
b_2 = np.array([1.,1.])

W0_3 = np.array([[0.5,0.6], [0.6,0.7], [0.3,0.2]])
b_3 = np.array([1.,1.,1.])

In [ ]:
xtrain = np.array([0.1,0.7,0.3])
ytrain = np.array([1,0,0])

In [ ]:
##Forward Pass
#HL1 ReLU
Z1 = relu(W0_1 @ xtrain+b_1)
#print(Z1)
#HL2 Sigmoid
Z2 = sigmoid(W0_2 @ Z1+b_2)
#print(Z2)
#OL Softmax
ypred = softmax(W0_3 @ Z2+b_3)
print(ypred)
print("Loss: ",-ytrain@np.log(ypred))

[0.35985276 0.43045902 0.20968822]
Loss:  1.0220603290496275


$\begin{align}
  \frac{\partial\mathcal{L}}{\partial b^{(3)}_{k}}&=\sum_{i}\frac{\partial\mathcal{L}}{\partial Z^{(3)}_{i}}\frac{\partial Z^{(3)}_{i}}{\partial b^{(3)}_{k}}\\
  &=\sum_{i}\sum_{j}\frac{\partial\mathcal{L}}{\partial Z^{(3)}_{i}}\frac{\partial Z^{(3)}_{i}}{\partial I^{(3)}_{j}}\frac{\partial I^{(3)}_{j}}{\partial b^{(3)}_{k}}\\
  &=\sum_{i}\sum_{j}-\frac{Y_{i}}{Z^{(3)}_{i}}\left(Z_{i}^{(3)}(\delta_{ij}-Z^{(3)}_{j})\right)(\delta_{jk})\\
  &=Z^{(3)}_{k}-Y_{k}\\
  \\
  \frac{\partial\mathcal{L}}{\partial w^{(3)}_{ij}}&=\sum_{k}\frac{\partial\mathcal{L}}{\partial Z^{(3)}_{k}}\frac{\partial Z^{(3)}_{k}}{\partial w^{(3)}_{ij}}\\
  &=\sum_{k}\sum_{l}\frac{\partial\mathcal{L}}{\partial Z^{(3)}_{k}}\frac{\partial Z^{(3)}_{k}}{\partial I^{(3)}_{l}}\frac{\partial I^{(3)}_{l}}{\partial w^{(3)}_{ij}}\\
  &=\sum_{k}\sum_{l}-\frac{Y_{k}}{Z^{(3)}_{k}}\left(Z_{k}^{(3)}(\delta_{kl}-Z^{(3)}_{l})\right)(\delta_{il}Z_{j}^{(2)})\\
  &=\sum_{k}\sum_{l}-Y_{k}\left(\delta_{kl}-Z^{(3)}_{l}\right)(\delta_{il}Z_{j}^{(2)})\\
  &=Z_{j}^{(2)}\sum_{k}-Y_{k}\left(\delta_{ik}-Z^{(3)}_{i}\right)\\
  &=Z_{j}^{(2)}(Z_{i}^{(3)}-Y_{i})\\
  \\
\frac{\partial\mathcal{L}}{\partial Z^{(2)}_{i}}&=\sum_{j}\frac{\partial\mathcal{L}}{\partial Z^{(3)}_{j}}\frac{\partial Z^{(3)}_{j}}{\partial Z^{(2)}_{i}}\\
  &=\sum_{j}\sum_{k}\frac{\partial\mathcal{L}}{\partial Z^{(3)}_{j}}\frac{\partial Z^{(3)}_{j}}{\partial I^{(3)}_{k}}\frac{\partial I^{(3)}_{k}}{\partial Z^{(2)}_{i}}\\
  &=\sum_{j}\sum_{k}-\frac{Y_{j}}{Z^{(3)}_{j}}\left(Z_{j}^{(3)}(\delta_{jk}-Z^{(3)}_{k})\right)(w^{(3)}_{ki})\\
  &=\sum_{j}\sum_{k}-\frac{Y_{j}}{Z^{(3)}_{j}}\left(Z_{j}^{(3)}(\delta_{jk}-Z^{(3)}_{k})\right)(w^{(3)}_{ki})\\
\end{align}$

so we have

$\begin{align}
\nabla_{b^{(3)}}\mathcal{L}&=\delta^{(3)}=Z^{(3)}-Y\\
\nabla_{W^{(3)}}\mathcal{L}&=\delta^{(3)}\otimes Z^{(2)}\\
\nabla_{b^{(2)}}\mathcal{L}&=\delta^{(2)}=(\nabla_{I^{(2)}}\sigma^{(2)})(W^{(3)})^{\intercal}\delta^{(3)}\\
&=Z^{(2)}\circ(1-Z^{(2)})\circ(W^{(3)})^{\intercal}\delta^{(3)}\\
\nabla_{W^{(2)}}\mathcal{L}&=\delta^{(2)}\otimes Z^{(1)}\\
\nabla_{b^{(1)}}\mathcal{L}&=\delta^{(1)}=(\nabla_{I^{(1)}}\sigma^{(1)})(W^{(2)})^{\intercal}\delta^{(2)}\\
&=H(I^{(1)})\circ(W^{(2)})^{\intercal}\delta^{(2)}\\
\nabla_{W^{(2)}}\mathcal{L}&=\delta^{(1)}\otimes X
\end{align}
$

In [ ]:
##Backward pass
delta3 = ypred-ytrain
dLdW3 = np.outer(delta3,Z2)
print(delta3,dLdW3)
delta2 = Z2*(1-Z2)*(W0_3.T @ delta3)
dLdW2 = np.outer(delta2,Z1)
print(delta2,dLdW2)
delta1 = np.heaviside(W0_1@xtrain+b_1,1)*(W0_2.T @ delta2)
dLdW1 = np.outer(delta1,xtrain)
print(delta1,dLdW1)

b_3-=delta3
W0_3-=dLdW3
b_2-=delta2
W0_2-=dLdW2
b_1-=delta1
W0_1-=dLdW1

print("Updated Weights and Biases")

print(W0_1,b_1)
print(W0_2,b_2)
print(W0_3,b_3)

[-0.64014724  0.43045902  0.20968822] [[-0.56510762 -0.58176209]
 [ 0.37999957  0.39119865]
 [ 0.18510806  0.19056344]]
[ 0.00011468 -0.00338424] [[ 0.000164    0.00017088]
 [-0.00483946 -0.00504251]]
[-0.00232309 -0.00064244] [[-2.32309311e-04 -1.62616517e-03 -6.96927932e-04]
 [-6.42442430e-05 -4.49709701e-04 -1.92732729e-04]]
Updated Weights and Biases
[[0.10023231 0.30162617 0.70069693]
 [0.90006424 0.40044971 0.40019273]] [1.00232309 1.00064244]
[[0.399836   0.29982912]
 [0.70483946 0.20504251]] [0.99988532 1.00338424]
[[1.06510762 1.18176209]
 [0.22000043 0.30880135]
 [0.11489194 0.00943656]] [1.64014724 0.56954098 0.79031178]


In [ ]:
k=1
while(k<201):
  k+=1
  print("Iteration: ", k)
  ##Forward Pass
  #HL1 ReLU
  Z1 = relu(W0_1 @ xtrain+b_1)
  #print(Z1)
  #HL2 Sigmoid
  Z2 = sigmoid(W0_2 @ Z1+b_2)
  #print(Z2)
  #OL Softmax
  ypred = softmax(W0_3 @ Z2+b_3)
  print(ypred)
  print("Loss: ",-ytrain@np.log(ypred))

  ##Backward pass
  delta3 = ypred-ytrain
  dLdW3 = np.outer(delta3,Z2)
  #print(delta3,dLdW3)
  delta2 = Z2*(1-Z2)*(W0_3.T @ delta3)
  dLdW2 = np.outer(delta2,Z1)
  #print(delta2,dLdW2)
  delta1 = np.heaviside(W0_1@xtrain+b_1,1)*(W0_2.T @ delta2)
  dLdW1 = np.outer(delta1,xtrain)
  #print(delta1,dLdW1)

  b_3-=delta3
  W0_3-=dLdW3
  b_2-=delta2
  W0_2-=dLdW2
  b_1-=delta1
  W0_1-=dLdW1

  print("Updated Weights and Biases")

  print(W0_1,b_1)
  print(W0_2,b_2)
  print(W0_3,b_3)

Iteration:  2
[0.87954341 0.06457517 0.05588142]
Loss:  0.12835236020870527
Updated Weights and Biases
[[0.10137754 0.30964278 0.70413262]
 [0.90060169 0.40421181 0.40180506]] [1.0137754  1.00601688]
[[0.41579591 0.3164272 ]
 [0.71908065 0.21985315]] [1.01101734 1.01331745]
[[ 1.17145869  1.29143659]
 [ 0.16298704  0.25000631]
 [ 0.06555426 -0.0414429 ]] [1.76060383 0.50496581 0.73443036]
Iteration:  3
[0.92228519 0.04126648 0.03644833]
Loss:  0.08090078793434305
Updated Weights and Biases
[[0.10221891 0.31553238 0.70665673]
 [0.90101094 0.40707657 0.40303281]] [1.02218911 1.01010938]
[[0.42745461 0.32846864]
 [0.72932741 0.2304363 ]] [1.01904728 1.02037493]
[[ 1.24061327  1.36261042]
 [ 0.12626604  0.21221308]
 [ 0.03312069 -0.0748235 ]] [1.83831864 0.46369933 0.69798203]
Iteration:  4
[0.94220177 0.03053017 0.02726806]
Loss:  0.05953583479959617
Updated Weights and Biases
[[0.10288945 0.32022618 0.70866836]
 [0.90134475 0.40941325 0.40403425]] [1.02889454 1.0134475 ]
[[0.43669346 0.3

Comparing to Tensorflow

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [ ]:
W01 = np.array([[0.1,0.3,0.7], [0.9,0.4,0.4]])
b1 = np.array([1.,1.])

W02 = np.array([[0.4,0.3], [0.7,0.2]])
b2 = np.array([1.,1.])

W03 = np.array([[0.5,0.6], [0.6,0.7], [0.3,0.2]])
b3 = np.array([1.,1.,1.])

In [ ]:
# Create the model
model = Sequential()
# Add the first hidden layer
model.add(Dense(2, input_dim=3, activation='relu', weights = [W01.T, b1]))
# Second hidden layer
model.add(Dense(2, activation='sigmoid', weights = [W02.T, b2]))
# Output layer
model.add(Dense(3, activation='softmax', weights = [W03.T, b3]))

sgd = SGD(lr=1)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['categorical_crossentropy'])
model.get_weights()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[array([[0.1, 0.9],
        [0.3, 0.4],
        [0.7, 0.4]], dtype=float32),
 array([1., 1.], dtype=float32),
 array([[0.4, 0.7],
        [0.3, 0.2]], dtype=float32),
 array([1., 1.], dtype=float32),
 array([[0.5, 0.6, 0.3],
        [0.6, 0.7, 0.2]], dtype=float32),
 array([1., 1., 1.], dtype=float32)]

In [ ]:
model.fit(xtrain.reshape((1,3)), ytrain.reshape((1, 3)), epochs=200, batch_size=1)

Epoch 1/200
1/1 [==============================] - 1s 540ms/step - loss: 1.0221 - categorical_crossentropy: 1.0221
Epoch 2/200
1/1 [==============================] - 0s 5ms/step - loss: 0.1284 - categorical_crossentropy: 0.1284
Epoch 3/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0809 - categorical_crossentropy: 0.0809
Epoch 4/200
1/1 [==============================] - 0s 11ms/step - loss: 0.0595 - categorical_crossentropy: 0.0595
Epoch 5/200
1/1 [==============================] - 0s 10ms/step - loss: 0.0472 - categorical_crossentropy: 0.0472
Epoch 6/200
1/1 [==============================] - 0s 11ms/step - loss: 0.0392 - categorical_crossentropy: 0.0392
Epoch 7/200
1/1 [==============================] - 0s 7ms/step - loss: 0.0335 - categorical_crossentropy: 0.0335
Epoch 8/200
1/1 [==============================] - 0s 8ms/step - loss: 0.0292 - categorical_crossentropy: 0.0292
Epoch 9/200
1/1 [==============================] - 0s 11ms/step - loss: 0.0259 - categorica

In [ ]:
model.predict(xtrain.reshape((1, 3)))

array([[9.9887174e-01, 5.7525293e-04, 5.5299088e-04]], dtype=float32)

In [ ]:
model.get_weights()

[array([[0.11550441, 0.9084708 ],
        [0.40853125, 0.4592939 ],
        [0.74651366, 0.4254116 ]], dtype=float32),
 array([1.1550443, 1.084706 ], dtype=float32),
 array([[0.6014873 , 0.87786746],
        [0.50195366, 0.3785293 ]], dtype=float32),
 array([1.1302494, 1.1153164], dtype=float32),
 array([[ 2.0983257 , -0.3175912 , -0.3807338 ],
        [ 2.2364383 , -0.24000825, -0.49643087]], dtype=float32),
 array([ 2.7668118 , -0.01687927,  0.25006643], dtype=float32)]

Hey! Lookie there! They're the same!